<a href="https://colab.research.google.com/github/cherrydaria/RecoServiceTemplate/blob/HW3/notebooks/User_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone -b HW3 https://github.com/cherrydaria/RecoServiceTemplate.git

fatal: destination path 'RecoServiceTemplate' already exists and is not an empty directory.


In [1]:
!pip install rectools

In [2]:
%cd RecoServiceTemplate
%cd service
from userknn import UserKnn

/content/RecoServiceTemplate
/content/RecoServiceTemplate/service


In [3]:
%cd ..
%cd functions
from my_metrics import *
from my_visualization import *

/content/RecoServiceTemplate
/content/RecoServiceTemplate/functions


In [4]:
import pandas as pd
import numpy as np
import zipfile
import requests
import io
from tqdm import tqdm

In [5]:
from pprint import pprint
from copy import deepcopy
import pickle
import os
from scipy.stats import mode
import warnings

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 200)

In [6]:
from rectools import Columns
from rectools.dataset import Dataset, Interactions
from rectools.metrics import MAP, NDCG, MeanInvUserFreq, Precision, Recall, Serendipity
from rectools.metrics.base import MetricAtK
from rectools.model_selection import Splitter, TimeRangeSplitter
from rectools.models import PopularModel, RandomModel
from rectools.models.base import ModelBase
from rectools.metrics import calc_metrics

from implicit.nearest_neighbours import CosineRecommender, TFIDFRecommender, BM25Recommender

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Проверка подключения к диску

In [8]:
file_path = '/content/drive/My Drive/example2.txt'

# Открываем файл для записи
with open(file_path, 'w') as file:
    file.write('Пример текста для записи в файл.')

print('Файл успешно создан и сохранен на Google Диске по пути:', file_path)

Файл успешно создан и сохранен на Google Диске по пути: /content/drive/My Drive/example2.txt


# Подготовка набора данных

In [9]:
url = 'https://github.com/irsafilo/KION_DATASET/raw/f69775be31fa5779907cf0a92ddedb70037fb5ae/data_original.zip'
req = requests.get(url, stream=True)

with open('kion_train.zip', "wb") as fd:
    total_size_in_bytes = int(req.headers.get('Content-Length', 0))
    progress_bar = tqdm(desc='Downloading the kion dataset...',
                        total=total_size_in_bytes,
                        unit='iB', unit_scale=True)
    for chunk in req.iter_content(chunk_size=2 ** 20):
        progress_bar.update(len(chunk))
        fd.write(chunk)

In [16]:
!unzip kion_train.zip -x '__MACOSX/*'

Archive:  kion_train.zip
replace data_original/interactions.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [11]:
interactions = pd.read_csv('data_original/interactions.csv')
users = pd.read_csv('data_original/users.csv')
items = pd.read_csv('data_original/items.csv')

In [12]:
interactions.rename(columns={'last_watch_dt': Columns.Datetime, 'total_dur': Columns.Weight}, inplace=True)

In [13]:
interactions['datetime'] = pd.to_datetime(interactions['datetime'])

Мини-датасет взаимодействий для проведения экспериментов

In [61]:
interactions_mini = interactions.sample(frac=0.1)

In [62]:
print(f"Interactions shape {interactions_mini.shape}")
print(f"Unique users in interactions: {interactions_mini['user_id'].nunique():_}")
print(f"Unique items in interactions: {interactions_mini['item_id'].nunique():_}")
print(f"min date in interactions: {interactions_mini['datetime'].min()}")
print(f"max date in interactions: {interactions_mini['datetime'].max()}")

Interactions shape (547625, 5)
Unique users in interactions: 307_112
Unique items in interactions: 9_946
min date in interactions: 2021-03-13 00:00:00
max date in interactions: 2021-08-22 00:00:00


Полный датасет взаимодейсвий для обучения модели knn и popular

In [17]:
pd.concat([interactions.head(), interactions.tail()])

,user_id,item_id,datetime,weight,watched_pct
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0
3,864613,7638,2021-07-05,14483,100.0
4,964868,9506,2021-04-30,6725,100.0
5476246,648596,12225,2021-08-13,76,0.0
5476247,546862,9673,2021-04-13,2308,49.0
5476248,697262,15297,2021-08-20,18307,63.0
5476249,384202,16197,2021-04-19,6203,100.0
5476250,319709,4436,2021-08-15,3921,45.0


In [18]:
print(f"Interactions shape {interactions.shape}")
print(f"Unique users in interactions: {interactions['user_id'].nunique():_}")
print(f"Unique items in interactions: {interactions['item_id'].nunique():_}")
print(f"min date in interactions: {interactions['datetime'].min()}")
print(f"max date in interactions: {interactions['datetime'].max()}")

Interactions shape (5476251, 5)
Unique users in interactions: 962_179
Unique items in interactions: 15_706
min date in interactions: 2021-03-13 00:00:00
max date in interactions: 2021-08-22 00:00:00


Users

In [19]:
pd.concat([users.head(), users.tail()])

,user_id,age,income,sex,kids_flg
0,973171,age_25_34,income_60_90,М,1
1,962099,age_18_24,income_20_40,М,0
2,1047345,age_45_54,income_40_60,Ж,0
3,721985,age_45_54,income_20_40,Ж,0
4,704055,age_35_44,income_60_90,Ж,0
840192,339025,age_65_inf,income_0_20,Ж,0
840193,983617,age_18_24,income_20_40,Ж,1
840194,251008,NaN,NaN,NaN,0
840195,590706,NaN,NaN,Ж,0
840196,166555,age_65_inf,income_20_40,Ж,0


In [20]:
print(f"Users shape {users.shape}")
print(f"Unique users: {users['user_id'].nunique():_}")

Users shape (840197, 5)
Unique users: 840_197


Items

In [21]:
pd.concat([items.head(2), items.tail(2)])

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Грандинетти, Джеральдин Чаплин, Елена Анайя, Каэтано Велозо, Леонор Уотлинг, Лола Дуэньяс, Лолес Леон, Малу Айродо, Мариола Фуэнтес, Пас Вега, Пина Бауш, Ро...",Мелодрама легендарного Педро Альмодовара «Поговори с ней» в 2003 году получила премию «Оскар» за лучший сценарий. Журналист Марко берет интервью у знаменитой женщины-тореро Лидии и вскоре влюбляе...,"Поговори, ней, 2002, Испания, друзья, любовь, сильные, женщины, преодоление, трудностей, отношения, дружба, отношения, паре, отношения, мужчины, женщины, романтические, отношения, потеря, близких,..."
1,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон Манцукас, Джон Глейсер, Карл Грин, Кристен Риттер, Лэнс Реддик, Морис Комт, Патрик Кернс, Ребекка Коллинз, Роза Салазар, Росс П. Кук, Стеффи Гроут, Ти...","Уморительная современная комедия на популярную тему о том, как не надо отмечать мальчишник. Главный герой усвоил, что не надо звать на свадьбу своего друга Джейсона, из-за которого он вместо сваде...","Голые, перцы, 2014, США, друзья, свадьбы, преодоление, трудностей, расставания, отношения, дружба, риск, недоразумение, мужская, дружба, мальчишники, девичники"
15961,4538,series,Среди камней,Darklands,2019.0,"драмы, спорт, криминал",Россия,0.0,18.0,NaN,"Марк О’Коннор, Конор МакМахон","Дэйн Уайт О’Хара, Томас Кэйн-Бирн, Джудит Родди, Марк О’Халлоран, Джимми Смоллхорн","Семнадцатилетний Дэмиен мечтает вырваться за пределы своего района и стать профессиональным бойцом. Когда его кумир и старший брат исчезает, парень попадает в чуждый ему мир насилия, наркотиков и ...","Среди, камней, 2019, Россия"
15962,3206,series,Гоша,NaN,2019.0,комедии,Россия,0.0,16.0,NaN,Михаил Миронов,"Мкртыч Арзуманян, Виктория Рунцова","Добродушный Гоша не может выйти из дома, чтобы не попасть в нелепую и курьёзную историю. Но даже неудачники мечтают о любви, и наш герой — не исключение, ведь оптимизма ему не занимать.","Гоша, 2019, Россия"


In [22]:
print(f"Items dataframe shape {items.shape}")
print(f"Unique item_id: {items['item_id'].nunique():_}")

Items dataframe shape (15963, 14)
Unique item_id: 15_963


# Разделение на train и test выборки

В этом пункте будет разделение на выборки для обучения User KNN модели и модели Popular на полном датасете взаимодействий.

In [23]:
max_date = interactions['datetime'].max()

# Разделение выборки по времени
train = interactions[(interactions['datetime'] < max_date - pd.Timedelta(days=7))]
test = interactions[(interactions['datetime'] >= max_date - pd.Timedelta(days=7))]

test_warm = test[test['user_id'].isin(train['user_id'].unique())]  # Теплые юзеры в тесте
test_cold = test[~test['user_id'].isin(train['user_id'].unique())]  # Холодные юзеры в тесте

print(f"train: {train.shape}")
print(f"test: {test.shape}")
print(f"test_warm: {test_warm.shape}")
print(f"test_cold: {test_cold.shape}")

train: (4985269, 5)
test: (490982, 5)
test_warm: (349088, 5)
test_cold: (141894, 5)


# Популярное

## Популярное по категории

In [24]:
# Разбиваем значения столбца "genres" на списки строк и создаем новый столбец "genre"
items["genre"] = items["genres"].str.split(",")

# Выводим первые строки столбцов "genre" и "genres" для просмотра
items[["genre", "genres"]].head()

,genre,genres
0,"[драмы, зарубежные, детективы, мелодрамы]","драмы, зарубежные, детективы, мелодрамы"
1,"[зарубежные, приключения, комедии]","зарубежные, приключения, комедии"
2,"[криминал, зарубежные, триллеры, боевики, комедии]","криминал, зарубежные, триллеры, боевики, комедии"
3,"[драмы, зарубежные, мелодрамы]","драмы, зарубежные, мелодрамы"
4,"[драмы, спорт, советские, мелодрамы]","драмы, спорт, советские, мелодрамы"


In [25]:
# Разбиваем DataFrame items на строки для каждого жанра и присваиваем каждому жанру item_id
genre_feature = items[["item_id", "genre"]].explode("genre")

# Переименовываем столбцы для удобства использования
genre_feature.columns = ["id", "value"]

# Создаем столбец "feature" и заполняем его значением "genre"
genre_feature["feature"] = "genre"

# Выводим первые строки DataFrame genre_feature
genre_feature.head()

,id,value,feature
0,10711,драмы,genre
0,10711,зарубежные,genre
0,10711,детективы,genre
0,10711,мелодрамы,genre
1,2508,зарубежные,genre


Отфильтруем genre_feature по тестовой выборке

In [26]:
genre_feature = genre_feature[genre_feature['id'].isin(train['item_id'])]

 Создаем датасет, используя информацию о взаимодействиях (interactions), отсутствии особенностей пользователей (None), информацию о жанрах элементов (genre_feature) и список категориальных особенностей элементов (['genre'])

In [32]:
dataset_feature = Dataset.construct(
    interactions_df=interactions,  # полный датасет, так как иначе не получается деоать предикты для новых юзеров последней недели
    user_features_df=None,
    item_features_df=genre_feature,
    cat_item_features=['genre']
)

Инициализация и обучение модели.

In [33]:
from rectools.models.popular_in_category import PopularInCategoryModel

In [39]:
%time

pop_cat = PopularInCategoryModel(category_feature='genre', n_categories=5)
pop_cat.fit(dataset_feature);

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.11 µs


Получим рекомендации для холодных юзеров из тестовой выборки

In [35]:
dataset = Dataset.construct(
    interactions_df=test_cold,
    user_features_df=None,
    item_features_df=None
)

In [40]:
%time

pop_recs_cat = pop_cat.recommend(
    dataset.user_id_map.external_ids,
    dataset=dataset_feature,
    k=10,
    filter_viewed=False
)

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 8.34 µs


Инициализация и обучение модели. Поменяем параметры mixing_strategy и ratio_strategy с дефолтных значений на group и proportional соответственно. Первое сгруппирует элементы каждой кадегории, второе позволит категориям получать пропорциональных их популярности доли в рекомендациях.

In [37]:
pop_cat_par = PopularInCategoryModel(
    category_feature='genre',
    n_categories=5,
    mixing_strategy='group',
    ratio_strategy='proportional'
    )
pop_cat_par.fit(dataset_feature);

In [38]:
pop_recs_cat_par = pop_cat_par.recommend(
    dataset.user_id_map.external_ids,
    dataset=dataset_feature,
    k=10,
    filter_viewed=False
)

Посмотрим результаты

In [41]:
pop_recs_cat = pop_recs_cat.merge(items[['item_id', 'title']],
                                   on='item_id',
                                   how='left')

pop_recs_cat_par = pop_recs_cat_par.merge(items[['item_id', 'title']],
                                   on='item_id',
                                   how='left')

In [45]:
pd.concat([
    pop_recs_cat.head(10),
    pop_recs_cat_par.head(10),
], axis=1)

,user_id,item_id,score,rank,title,user_id,item_id,score,rank,title
0,505244,15297,193123.0,1,Клиника счастья,505244,15297,193123.0,1,Клиника счастья
1,505244,9728,132865.0,2,Гнев человеческий,505244,13865,122119.0,2,Девятаев
2,505244,12192,38242.0,3,Фемида видит,505244,2657,68581.0,3,Подслушано
3,505244,7829,20017.0,4,Поступь хаоса,505244,142,45367.0,4,Маша
4,505244,849,13713.0,5,Дебошир,505244,9728,132865.0,5,Гнев человеческий
5,505244,13865,122119.0,6,Девятаев,505244,12192,38242.0,6,Фемида видит
6,505244,7571,28372.0,7,100% волк,505244,7571,28372.0,7,100% волк
7,505244,2657,68581.0,8,Подслушано,505244,1844,24375.0,8,Аферистка
8,505244,1844,24375.0,9,Аферистка,505244,7829,20017.0,9,Поступь хаоса
9,505244,142,45367.0,10,Маша,505244,849,13713.0,10,Дебошир


Видим, что в принципе рекомендации одни и те же, однако ранг различается.

# Эксперименты

In [70]:
# Путь к файлу для сохранения результатов
file_path = '/content/drive/My Drive/results_cv_2.pkl'

Здесь используем датасет *interactions_mini*

Конечно, подсчет на мини датасете не даст точных значений метрик, только примерные, однако нет другого выхода, так каак мне не хватает скорости и мощностей, а тратить на эксперименты 20 часов...

In [74]:
interactions_mini = interactions.sample(frac=0.3)

In [75]:
interactions_mini.head()

,user_id,item_id,datetime,weight,watched_pct
811257,76765,3734,2021-07-19,202,4.0
4273575,851626,10073,2021-06-23,10,0.0
1222977,622175,8139,2021-05-26,359,6.0
3805971,1036448,15297,2021-06-22,21860,38.0
2310920,448794,7444,2021-07-16,5601,94.0


In [76]:
interactions_mini_df = interactions_mini
interactions_mini = Interactions(interactions_mini)

Фолды для кросс валидации

In [77]:
N_SPLITS = 3
TEST_SIZE = '7D'

In [78]:
# Init generator of folds
cv = TimeRangeSplitter(
    test_size=TEST_SIZE,
    n_splits=N_SPLITS,
    filter_already_seen=True,
    filter_cold_items=True,
    filter_cold_users=True,
)

In [79]:
cv.get_test_fold_borders(interactions_mini)

[(Timestamp('2021-08-02 00:00:00'), Timestamp('2021-08-09 00:00:00')),
 (Timestamp('2021-08-09 00:00:00'), Timestamp('2021-08-16 00:00:00')),
 (Timestamp('2021-08-16 00:00:00'), Timestamp('2021-08-23 00:00:00'))]

ЗАдаем метрики и модели для CV

In [80]:
# calculate several classic (precision@k and recall@k) and "beyond accuracy" metrics
metrics = {
    'precision@10': Precision(k=10),
    'recall@10': Recall(k=10),
    'map@10': MAP(k=10),
    'novelty': MeanInvUserFreq(k=10),
    'serendipity@10': Serendipity(k=10)
}

# few simple models to compare
models = {
    'cosine_userknn': CosineRecommender(), # implicit
    'tfidf_userknn': TFIDFRecommender(),
}

In [54]:
import pickle

In [81]:
%%time

results = []

fold_iterator = cv.split(interactions_mini, collect_fold_stats=True)

for i_fold, (train_ids, test_ids, fold_info) in enumerate(fold_iterator):
    print(f"\n==================== Fold {i_fold}")
    pprint(fold_info)

    df_train = interactions_mini.df.iloc[train_ids].copy()
    df_test = interactions_mini.df.iloc[test_ids][Columns.UserItem].copy()

    catalog = df_train[Columns.Item].unique()

    for model_name, model in models.items():
        userknn_model = UserKnn(model=model, N_users=10)  # 10 соседей
        userknn_model.fit(df_train)

        recos = userknn_model.predict(df_test)

        metric_values = calc_metrics(
            metrics,
            reco=recos,
            interactions=df_test,
            prev_interactions=df_train,
            catalog=catalog,
        )

        fold = {"fold": i_fold, "model": model_name}
        fold.update(metric_values)
        results.append(fold)

    with open(file_path, 'wb') as dumpfile:  # запись результатов в файл на гугл диске
        pickle.dump(results, dumpfile)


==================== Fold 0
{'end': Timestamp('2021-08-09 00:00:00'),
 'i_split': 0,
 'start': Timestamp('2021-08-02 00:00:00'),
 'test': 64577,
 'test_items': 4392,
 'test_users': 39137,
 'train': 1279952,
 'train_items': 12136,
 'train_users': 471045}


  0%|          | 0/471045 [00:00<?, ?it/s]

  0%|          | 0/471045 [00:00<?, ?it/s]


==================== Fold 1
{'end': Timestamp('2021-08-16 00:00:00'),
 'i_split': 1,
 'start': Timestamp('2021-08-09 00:00:00'),
 'test': 69312,
 'test_items': 4641,
 'test_users': 41676,
 'train': 1394478,
 'train_items': 12364,
 'train_users': 504863}


  0%|          | 0/504863 [00:00<?, ?it/s]

  0%|          | 0/504863 [00:00<?, ?it/s]


==================== Fold 2
{'end': Timestamp('2021-08-23 00:00:00'),
 'i_split': 2,
 'start': Timestamp('2021-08-16 00:00:00'),
 'test': 74479,
 'test_items': 4690,
 'test_users': 44586,
 'train': 1515289,
 'train_items': 12558,
 'train_users': 539572}


  0%|          | 0/539572 [00:00<?, ?it/s]

  0%|          | 0/539572 [00:00<?, ?it/s]

CPU times: user 39min 24s, sys: 19.4 s, total: 39min 43s
Wall time: 27min 16s


Откроем файлик с результатми

In [82]:
with open(file_path, 'rb') as dumpfile:
    results = pickle.load(dumpfile)
results

[{'fold': 0,
  'model': 'cosine_userknn',
  'precision@10': 0.0008201957227176329,
  'recall@10': 0.005540205437361581,
  'map@10': 0.0012332357416384588,
  'novelty': 8.081387135929493,
  'serendipity@10': 7.154932632000954e-06},
 {'fold': 0,
  'model': 'tfidf_userknn',
  'precision@10': 0.0013950992666785908,
  'recall@10': 0.009601445290818517,
  'map@10': 0.0020974382141199076,
  'novelty': 8.464775165085895,
  'serendipity@10': 5.215134484440061e-06},
 {'fold': 1,
  'model': 'cosine_userknn',
  'precision@10': 0.0007870237066897014,
  'recall@10': 0.005228123539021898,
  'map@10': 0.0010601365144833122,
  'novelty': 8.082791913584176,
  'serendipity@10': 7.835974347530511e-06},
 {'fold': 1,
  'model': 'tfidf_userknn',
  'precision@10': 0.0012669162107687877,
  'recall@10': 0.00875330326273315,
  'map@10': 0.001809253713552151,
  'novelty': 8.480156634046995,
  'serendipity@10': 7.787207194416203e-06},
 {'fold': 2,
  'model': 'cosine_userknn',
  'precision@10': 0.000879199748800071

Метрики качества по фолдам

In [83]:
df_metrics = pd.DataFrame(results)
df_metrics

,fold,model,precision@10,recall@10,map@10,novelty,serendipity@10
0,0,cosine_userknn,0.000820,0.005540,0.001233,8.081387,0.000007
1,0,tfidf_userknn,0.001395,0.009601,0.002097,8.464775,0.000005
2,1,cosine_userknn,0.000787,0.005228,0.001060,8.082792,0.000008
3,1,tfidf_userknn,0.001267,0.008753,0.001809,8.480157,0.000008
4,2,cosine_userknn,0.000879,0.005499,0.001184,8.166236,0.000008
5,2,tfidf_userknn,0.001361,0.008993,0.001847,8.566763,0.000008


Среднее значение метрик

In [84]:
df_metrics.groupby('model').mean()[metrics.keys()]

,precision@10,recall@10,map@10,novelty,serendipity@10
model,,,,,
cosine_userknn,0.000829,0.005423,0.001159,8.110138,0.000008
tfidf_userknn,0.001341,0.009116,0.001918,8.503898,0.000007


**Выводы по метрикам**
- *precision* и *recall*: идеальное значение для этих метрик равно 1, однако же не удалось и близко подобраться даже к значению 0.1, произошло это по причине -- была задействована лишь треть датасета (до этого считала и для 1% и для 10% от исходного размера датасета, с увеличение размера, увеличивались и значения метрик). *По этим двум метрикам пока что лучше результат у TFIDF*
- *MAP*: идеальное значение 1, все по той же причине, описанной пунктом выше, не удалось подобраться к высокому значению метрики. *Лучше результат у TFIDF*
- *MeanInvUserFreq*: CosineRecommender имеет результат чуть меньше, чем TFIDF, то есть ненамного лучше.
- *Serendipity*: Чем больше результат неожиданности - тем лучше, по этой метрике побеждает CosineRecommender.

***TFIDF*** имеет более высокие показатели *precision*, *recall*, *MAP*, что указывает на более точные и полные рекомендации.
***TFID*** также обладает более низким значением *MeanInvUserFreq*, что означает, что его рекомендации более известны пользователям.
Оба метода имеют низкое значение *Serendipiyu*, что может указывать на то, что рекомендации достаточно предсказуемы.

***Для обучения модели на полных данных лучше использовать TFIDF***

# User KNN модель (обучение на полном датасете)

Датасет, с которым работаем в этом пункте:
- *interactions*

Тренировочная и тестовая выборки:
- train
- test_warm

In [86]:
# Путь к файлу для сохранения результатов
file_path = '/content/drive/My Drive/results_full.pkl'

Обучим модель

In [87]:
catalog = train[Columns.Item].unique()

In [89]:
userknn_full = UserKnn(model=TFIDFRecommender(K=20), N_users=20)
userknn_full.fit(train)

  0%|          | 0/896791 [00:00<?, ?it/s]

Сохраним модель

In [91]:
!pip install dill

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.8 MB/s eta 0:00:00


In [92]:
import dill

In [93]:
# Путь к файлу для сохранения результатов
model_path = '/content/drive/My Drive/userknn.dill'

In [94]:
with open(model_path, 'wb') as f:
    dill.dump(userknn_full, f)

In [95]:
with open(model_path, 'rb') as f:
    userknn_full = dill.load(f)

Запишем предикты для теплых юзеров из тестовой выборки

In [99]:
userknn_full_pred = userknn_full.predict(test_warm)

In [100]:
userknn_full_pred

,user_id,item_id,score,rank
0,1097544,6309,5.630064,1
2,1097544,12768,4.638858,2
6,1097544,11505,3.552132,3
7,1097544,15942,3.27097,4
5,1097544,4689,3.141126,5
...,...,...,...,...
3024482,3,3475,2.753352,7
3024462,3,6208,2.737224,8
3024475,3,4250,2.732974,9
3024474,3,9381,2.659801,10


In [101]:
userknn_full_pred_merged = userknn_full_pred.merge(items[['item_id', 'title']],
                                   on='item_id',
                                   how='left')

In [102]:
userknn_full_pred_merged

,user_id,item_id,score,rank,title
0,1097544,6309,5.630064,1,Карантин
1,1097544,12768,4.638858,2,Птицы
2,1097544,11505,3.552132,3,Бездомный Бог
3,1097544,15942,3.27097,4,Азнавур
4,1097544,4689,3.141126,5,Уиджа. Проклятое зеркало
...,...,...,...,...,...
871056,3,3475,2.753352,7,Игры дьявола
871057,3,6208,2.737224,8,Страна чудес Вилли
871058,3,4250,2.732974,9,Зеркала: Инкарнация
871059,3,9381,2.659801,10,Кто не спрятался


Посчитаем метрики

In [103]:
metric_values_full = calc_metrics(
            metrics,
            reco=userknn_full_pred,
            interactions=test_warm,
            prev_interactions=train,
            catalog=catalog,
        )

In [106]:
metric_values_full

{'precision@10': 0.006341738647018313,
 'recall@10': 0.03040963346013946,
 'map@10': 0.0058447523806030265,
 'novelty': 7.671744788118634,
 'serendipity@10': 6.924403564537138e-05}

Модель показывает довольно низкие значения метрик, особенно *precision* и *MAP*. *Recall* выше, но все равно остается на низком уровне. *Novelty* рекомендаций достаточно высока,Что отражает новизну и разнообразие созданных рекомендаций, но *serendipity* находится на очень низком уровне, что говорит нам о предсказуемости рекомендованного. Требуется дальнейшее улучшение модели для достижения лучших результатов по всем метрикам.